In [31]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import emoji

import re
    
    
%matplotlib inline

In [9]:
filepath = '/Users/alexandraplassaras/src/spark_joy/comments.json'

In [10]:
with open(filepath, 'r') as f:
    data = json.load(f)

In [11]:
comment_list = []
for item in data['items']:
    comment_row = item['snippet']['topLevelComment']['snippet']
    del comment_row['authorDisplayName']
    del comment_row['authorProfileImageUrl']
    del comment_row['canRate']
    del comment_row['updatedAt']
    del comment_row['viewerRating']
    del comment_row['textOriginal']
    del comment_row['authorChannelUrl']
    del comment_row['likeCount']

    comment_list.append(comment_row)


    
#     snippet = data['items'][0]['snippet']['topLevelComment']['snippet']

In [12]:
df = pd.DataFrame(comment_list)

In [53]:
df.head()


,authorChannelId,publishedAt,textDisplay,videoId,emoji
0,{'value': 'UCGWE5YRjoodVGyuIhy0MD1g'},2019-01-27T02:20:59.000Z,I&#39;m so happy that I found your channel. I&...,O6EruBH1v7M,[]
1,{'value': 'UCziUx7-BthhCbWInV0F6hfQ'},2019-01-26T23:52:01.000Z,"Thanks for this video, can’t wait to see the r...",O6EruBH1v7M,[]
2,{'value': 'UCs_t9Jdbwa0ZqnBaitvPanQ'},2019-01-26T23:16:09.000Z,Dude. My room is messier than the before picture,O6EruBH1v7M,[]
3,{'value': 'UCv7djtU-TXHmhp42rliuW7Q'},2019-01-26T23:14:36.000Z,👏😍😍👏👏😍👏😍👏👏😍 declutter is life 🤩🤩🤩🤩 and u are m...,O6EruBH1v7M,"[👏😍😍👏👏😍👏😍👏👏😍, 🤩🤩🤩🤩, 💕❤️💕❤️💕❤️💕❤️]"
4,{'value': 'UCG08L1bkWCIro2HrJTHB02A'},2019-01-26T20:57:18.000Z,This video was just soo satisfying 😍,O6EruBH1v7M,[😍]


In [36]:
def extract_emojis(sentence):
    return [word for word in sentence.split() if str(word.encode('unicode-escape'))[2] == '\\' ]

In [38]:
df['emoji'] = df['textDisplay'].apply(lambda x: extract_emojis(x))

In [54]:
df.head()

,authorChannelId,publishedAt,textDisplay,videoId,emoji
0,{'value': 'UCGWE5YRjoodVGyuIhy0MD1g'},2019-01-27T02:20:59.000Z,I&#39;m so happy that I found your channel. I&...,O6EruBH1v7M,[]
1,{'value': 'UCziUx7-BthhCbWInV0F6hfQ'},2019-01-26T23:52:01.000Z,"Thanks for this video, can’t wait to see the r...",O6EruBH1v7M,[]
2,{'value': 'UCs_t9Jdbwa0ZqnBaitvPanQ'},2019-01-26T23:16:09.000Z,Dude. My room is messier than the before picture,O6EruBH1v7M,[]
3,{'value': 'UCv7djtU-TXHmhp42rliuW7Q'},2019-01-26T23:14:36.000Z,👏😍😍👏👏😍👏😍👏👏😍 declutter is life 🤩🤩🤩🤩 and u are m...,O6EruBH1v7M,"[👏😍😍👏👏😍👏😍👏👏😍, 🤩🤩🤩🤩, 💕❤️💕❤️💕❤️💕❤️]"
4,{'value': 'UCG08L1bkWCIro2HrJTHB02A'},2019-01-26T20:57:18.000Z,This video was just soo satisfying 😍,O6EruBH1v7M,[😍]


In [42]:
emoji_lst = df['emoji'].tolist()

In [50]:
main_emoji = []
for i in emoji_lst:
    if i == []:
        continue
    for j in i:
        for k in j:
            main_emoji.append(k)

In [56]:
main_emoji[:5]

['👏', '😍', '😍', '👏', '👏']